# Data Regionalization Example 
v2025.06.02

# Setup

Run the following to get your virtual enviroment set up:

```sh
$ conda create -n geoc python=3.12
$ conda activate geoc
$ pip install openpyxl
$ pip install geopandas
$ pip install requests
$ pip install matplotlib
$ pip install jupyterlab
```

Most of the data files used in this script are automatically downloaded from their sources (using Python's `requests` or Pandas's `read_excel`).

The data file sources:

- Balancing Authority Areas
    - U.S. Energy Atlas ([.html](https://atlas.eia.gov/datasets/09550598922b429ca9f06b9a067257bd_255/explore))
- U.S. Census Tracts
    - U.S. Census Bureau TIGER Cartographic Boundary Files at 500k resolution (based on census year global parameter)
        - 2020 ([.zip](https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_tract_500k.zip))
- U.S. Counties
    - U.S. Census Bureau TIGER Cartographic Boundary Files at 500k resolution (based on census year global parameter)
        - 2020 ([.zip](https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_county_500k.zip))
- U.S. States
    - U.S. Census Bureau TIGER Cartographic Boundary Files at 500k resolution (based on census year global parameter)
        - 2020 ([.zip](https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_state_500k.zip))
- Coal Basins
    - EIA.gov ([.zip](https://www.eia.gov/maps/map_data/cbm_4shps.zip))
- Electricity Market Module Regions
    - EIA.gov ([.html](https://www.eia.gov/outlooks/aeo/additional_docs.php))
- NERC Subregions
    - EIA ([.html](https://services1.arcgis.com/Hp6G80Pky0om7QvQ/arcgis/rest/services/NERC_Regions/FeatureServer/0/)) 
- Natural Gas Basins
    - EPA's Office of Air and Radiation (OAP) and Office of Atmospheric Protection (OAP) ([.zip](https://edg.epa.gov/data/Public/OAR/OAP/Basins_Shapefile.zip)) 

# Methods

In [ ]:
# Import required packages, globals, and methods
import os
import re

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import regionalizer as rlyr

In [ ]:
# Setup inline matplotlib plotting parameters
%matplotlib inline

In [ ]:
# Set up the logger for convenience, as there are logging statements
# throughout the code to tell us what's going on.
logger = rlyr.get_logger()

In [ ]:
# Show starting and ending region options.
rlyr.show_start_extents()

In [ ]:
rlyr.show_end_extents()

## Runtime

Here, we can read and/or create the conversion matrices from our starting spatial extent&mdash;such as balancing authorities ('BA'), coal basins ('CB'), or natural gas basins ('NG')&mdash;to census tract.

The weighting options are 'A' for areal and 'Eq' for equal.
Use 'US' as the state to run for the whole U.S.

In [ ]:
print("Census year:", rlyr.CENSUS_YEAR)
print("BA year:", rlyr.BA_YEAR)

In [ ]:
M, codes, cen_codes = rlyr.get_m(start_extent='BA', end_extent='CT', weighting='A', state='US')

## Testing

The conversion algorithm is:

$$
c = a \times M
$$

where `a` is a vector of our input geometry's attributes (e.g., GWP amounts for balancing authorities), `M` is our conversion matrix, and `c` is the vector of our input geometry's attributes converted to census tract areas.

We've already created `M`.
Let's create `a`.

Recall that the order of input geometries matters and must match the names given in `codes` list.

### Prepare the data
In this example, we'll use GWP (IPCC AR5 100-year, kg CO2e/MWh delivered) by BA based on the consumption mix results taken from NETL's GridMixExplorer v4.2 ([.html](https://netl.doe.gov/energy-analysis/details?id=f0f94954-3627-4e9b-a5c0-c29cfe419d1c)).
These results are saved in the inputs directory as "BA_GWP.csv".

In [ ]:
def cleanup_ba_names(x):
    p = re.compile("^(.*)\\s{1}\\(\\w+\\)$")
    r = p.search(x)
    if r:
        return r.group(1)
    else:
        return x


# Testing w/ BA-level impacts from NETL's GridMixExplorer 4.2
input_csv = os.path.join("inputs", "BA_GWP.csv")
if not os.path.isfile(input_csv):
    raise OSError("Failed to find, '%s'" % input_csv)

data_in = pd.read_csv(input_csv)

# Drop total column:
data_in = data_in.drop(columns='Total')

# Convert columns to rows
data_in = data_in.melt(
    id_vars=['Impact Category'],
    var_name="BA_NAME",
    value_name="Value"
)

# Drop impact category column
data_in = data_in.drop(columns='Impact Category')

# Remove the parenthetical, correct the all-caps, and gap-fill any mis-matches with original name
data_in['NAME'] = data_in['BA_NAME'].apply(cleanup_ba_names)
data_in = correct_ba_geo_names(data_in)
data_in.loc[data_in['BA_NAME'].isna(), 'BA_NAME'] = data_in.loc[data_in['BA_NAME'].isna(), 'NAME']

# Add the BA codes
data_in = map_ba_codes(data_in)

In [ ]:
data_in.loc[data_in['BA_CODE'].isna(), :]

In [ ]:
# Manual fixes to mis-matches
data_in.loc[data_in['BA_NAME'] == 'Arlington Valley, LLC - AVBA', 'BA_CODE'] = 'DEAA'
data_in.loc[data_in['BA_NAME'] == 'B.C. Hydro & Power Authority', 'BA_CODE'] = 'BCHA'
data_in.loc[data_in['BA_NAME'] == 'Homestead, City of', 'BA_CODE'] = 'HST'
data_in.loc[data_in['BA_NAME'] == 'NaturEner Power Watch, LLC (GWA)', 'BA_CODE'] = 'GWA'
data_in.loc[data_in['BA_NAME'] == 'New Harquahala Generating Company, LLC - HGBA', 'BA_CODE'] = 'HGBA'
data_in.loc[data_in['BA_NAME'] == 'New Smyrna Beach, Utilities Commission of', 'BA_CODE'] = 'NSB'
data_in.loc[data_in['BA_NAME'] == 'NORTHWESTERN ENERGY', 'BA_CODE'] = 'NWMT'
# This last one taken from DOE's BA lookup tool
# https://www.energy.gov/femp/balancing-authority-lookup-tool
data_in.loc[data_in['BA_NAME'] == 'South Carolina Electric & Gas Company', 'BA_CODE'] = 'SCEG'

In [ ]:
data_in

Convert units of kg CO2e/MWh delivered to kg CO2e by multiplying by the consumption, MWh.

$$
\mathrm{Consumption} = \mathrm{Net\, Generation} + \mathrm{Imports} - \mathrm{Exports}
$$


> The files `net_gen_sum_2016.csv` and `ba_trades_2016.csv` were generated using the development branch for version 2.0 of [ElectricityLCI](https://github.com/USEPA/ElectricityLCI/tree/development).
> The pseudo-code for recreating these CSV files is as follows.

1. From main.py, run imports (top matter), create root logger (in the if statement at the bottom of the model), and build model config for ELCI_1 (in main.main).
2. From eia_io_trading.py, import everything (all libraries, globals, and methods).
3. Find ba_io_trading_model method. Set `year`, `subregion`, and `regions_to_keep` variables to None. Execute all lines of code in this method down to (and including) the call to `qio_model`. Save two data frames to file:

```python
df_net_gen_sum.to_csv("net_gen_sum_2016.csv")
df_final_trade_out_filt_melted_merge[['export BAA', 'import BAA', 'value']].to_csv("ba_trades_2016.csv", index=False)
```

In [ ]:
# Define the net gen and trade CSV files; these were created in ElectricityLCI python package.
ng_csv = os.path.join("inputs", "net_gen_sum_2016.csv")
bt_csv = os.path.join("inputs", "ba_trades_2016.csv")

# Read files (correct column names)
ng_df = pd.read_csv(ng_csv)
ng_df.columns = ['BA_CODES', 'NET_GEN']
bt_df = pd.read_csv(bt_csv)

# Get total exports by BAA
ba_exp = bt_df.groupby(by='export BAA').agg({'value': 'sum'})
ba_exp = ba_exp.reset_index()

# Get total imports by BAA
ba_imp = bt_df.groupby(by="import BAA").agg({'value': 'sum'})
ba_imp = ba_imp.reset_index()

# Merge exports
cons_df = pd.merge(
    left=ng_df,
    right=ba_exp,
    left_on='BA_CODES',
    right_on='export BAA',
    how='left'
)
cons_df = cons_df.rename(columns={'value': 'Export'})

# Merge imports
cons_df = pd.merge(
    left=cons_df,
    right=ba_imp,
    left_on='BA_CODES',
    right_on='import BAA',
    how='left'
)
cons_df = cons_df.rename(columns={'value': 'Import'})

# Fill NaNs
cons_df['Export'] = cons_df['Export'].fillna(0)
cons_df['Import'] = cons_df['Import'].fillna(0)

# Calculate consumption, MWh
cons_df['Consumption'] = cons_df['NET_GEN'] - cons_df['Export'] + cons_df['Import']

In [ ]:
cons_df

In [ ]:
# Add consumption to GWP data frame
data_in = pd.merge(
    left=data_in,
    right=cons_df,
    left_on='BA_CODE',
    right_on='BA_CODES',
    how='left'
)
# NEW Correct any missing Consumption BAs
data_in['Consumption'] = data_in['Consumption'].fillna(0)

In [ ]:
# Multiply Value (kg CO2e/MWh delivered) by Consumption (MWh).
data_in['GWP'] = data_in['Value'] * data_in['Consumption']

In [ ]:
# Create the input array in the correct order.
fix_negatives = True
input_array = []
for ba_code in codes:
    tmp_df = data_in.query("BA_CODE == '%s'" % ba_code)
    if len(tmp_df) > 0:
        input_val = float(tmp_df['GWP'].values[0])
    else:
        input_val = 0.0

    # Add a check for negatives and fix
    if fix_negatives and input_val < 0:
        logger.info("Setting %f to zero for %s" % (input_val, ba_code))
        input_val = 0.0
    input_array.append(input_val)

### Run the calculation

In [ ]:
output_array = np.dot(input_array, M)

### Plot the results
In this section, we will add the BA-level and census-tract-level data to their respective geo data frames and then plot them.

In [ ]:
ba_geo = get_ba_geo(True, True)

In [ ]:
census_geo = get_census_geo(2020, 'us', True)

In [ ]:
# Add GWP to BA geodatabase
ba_geo['GWP'] = 0.0
for _, row in data_in.iterrows():
    # Get the BA code and GWP value
    ba_code = row['BA_CODE']
    gwp = row['GWP']
    # Search geodatabase for the BA code
    s_crit = ba_geo['BA_CODE'] == ba_code
    if s_crit.sum() > 0:
        ba_geo.loc[s_crit, 'GWP'] = gwp


In [ ]:
ba_geo.plot("GWP", cmap='plasma', legend=True, figsize=(8,4))

In [ ]:
# Save the BA-level data to GeoJSON---remember the CRS needs to be WGS84
to_save_ba = False
if to_save_ba:
    ba_outfile   = os.path.join("outputs", "ba_gwp.geojson")
    ba_wgs84 = ba_geo.to_crs(('epsg', 3857))
    ba_wgs84.to_file(ba_outfile, driver="GeoJSON")

In [ ]:
# Add transformed GWP to census tracts
# Remember the data order matches the cen_codes.
census_geo['GWP'] = 0.0
for idx, row in census_geo.iterrows():
    cen_code = row['GEOID']
    cen_idx = cen_codes.index(cen_code)
    cen_amt = output_array[cen_idx]
    census_geo.loc[idx, 'GWP'] = cen_amt

In [ ]:
census_geo.plot("GWP", cmap
                ='plasma', legend=True, figsize=(8,4))
# ct_png = os.path.join("outputs", "census_ba_a.png")
# plt.savefig(ct_png)

In [ ]:
# Save as GeoJSON---remember the CRS needs to be WGS84
to_save_ct = False
if to_save_ct:
    ct_outfile = os.path.join("outputs", "ct_ba_eq.geojson")
    census_wgs84 = census_geo.to_crs(('epsg', 3857))
    census_wgs84.to_file(ct_outfile, driver="GeoJSON")